In [2]:
%pip install instaloader


     ---------------------------------------- 68.3/68.3 kB 3.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [28]:

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys # 키보드 입력용
from webdriver_manager.chrome import ChromeDriverManager
import time
import os
import requests

# --- 설정 ---
HASHTAGS = ["석당박물관","동아대석당박물관","부산임시수도정부청사","구덕운동장","구덕야영장","구덕야영장잔디구장"]
TARGET_COUNT = 100 

# 브라우저 실행
options = webdriver.ChromeOptions()
# options.add_argument("--headless") # 절대 주석 풀지 마세요 (막힘)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

driver.get("https://www.instagram.com/")
print("\n" + "="*50)
print("⚠️  [필수] 브라우저에서 로그인 하세요.")
print("⚠️  [중요] 로그인 후 '정보 저장' 팝업이 뜨면 '나중에 하기'를 눌러 팝업을 없애세요.")
print("⚠️  준비되면 엔터(Enter)를 치세요.")
print("="*50)
input("waiting...") 

for tag in HASHTAGS:
    print(f"\n🔍 '{tag}' 수집 시작...")
    driver.get(f"https://www.instagram.com/explore/tags/{tag}/")
    time.sleep(5) 

    if not os.path.exists(tag):
        os.makedirs(tag)

    collected_count = 0
    visited_urls = set()
    consecutive_failures = 0 

    while collected_count < TARGET_COUNT:
        # 1. '더 보기' 버튼이 있으면 누르기 (가장 중요)
        try:
            more_btn = driver.find_element(By.CSS_SELECTOR, "div[role='button'] circle") # 더보기 아이콘
            if more_btn:
                driver.execute_script("arguments[0].click();", more_btn)
                time.sleep(2)
        except:
            pass

        # 2. 이미지 찾기 (조건 완화)
        images = driver.find_elements(By.TAG_NAME, "img")
        
        saved_in_this_turn = 0
        
        for img in images:
            if collected_count >= TARGET_COUNT:
                break
            
            try:
                img_url = img.get_attribute("src")
                
                # 최소한의 필터 (아이콘 제외)
                if not img_url or not img_url.startswith('http') or "static" in img_url:
                    continue
                
                # [수정] 크기 필터 삭제 (로딩 덜 된 이미지도 일단 가져옴)

                if img_url in visited_urls:
                    continue

                visited_urls.add(img_url)
                
                # 다운로드
                response = requests.get(img_url, timeout=5)
                filename = f"{tag}/{tag}_{collected_count+1}.jpg"
                with open(filename, "wb") as f:
                    f.write(response.content)
                
                collected_count += 1
                saved_in_this_turn += 1
                
                if collected_count % 10 == 0:
                    print(f"   [{collected_count}/{TARGET_COUNT}] 완료")

            except Exception:
                pass
        
        # 3. 진행 상황 체크
        if saved_in_this_turn == 0:
            consecutive_failures += 1
            print(f"   ⚠️ 새 이미지 로딩 중... ({consecutive_failures}/7)")
            
            # 스크롤을 살짝 올렸다가 다시 내림 (새로고침 유도)
            driver.execute_script("window.scrollBy(0, -200);")
            time.sleep(1)
            driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END) # 키보드 END키 전송
            time.sleep(5) # 대기 시간 늘림
        else:
            consecutive_failures = 0
            # 정상 진행 중이면 그냥 쭉 내림
            driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
            time.sleep(3)

        if consecutive_failures >= 7: # 기회 7번으로 늘림
            print("   ❌ 더 이상 이미지가 안 뜹니다. 다음 태그로 이동.")
            break

print("\n🎉 종료되었습니다.")
driver.quit()


⚠️  [필수] 브라우저에서 로그인 하세요.
⚠️  [중요] 로그인 후 '정보 저장' 팝업이 뜨면 '나중에 하기'를 눌러 팝업을 없애세요.
⚠️  준비되면 엔터(Enter)를 치세요.

🔍 '석당박물관' 수집 시작...
   [10/100] 완료
   [20/100] 완료
   [30/100] 완료
   ⚠️ 새 이미지 로딩 중... (1/7)
   ⚠️ 새 이미지 로딩 중... (1/7)
   ⚠️ 새 이미지 로딩 중... (2/7)
   ⚠️ 새 이미지 로딩 중... (3/7)
   ⚠️ 새 이미지 로딩 중... (4/7)
   ⚠️ 새 이미지 로딩 중... (5/7)
   ⚠️ 새 이미지 로딩 중... (6/7)
   ⚠️ 새 이미지 로딩 중... (7/7)
   ❌ 더 이상 이미지가 안 뜹니다. 다음 태그로 이동.

🔍 '동아대석당박물관' 수집 시작...
   [10/100] 완료
   [20/100] 완료
   ⚠️ 새 이미지 로딩 중... (1/7)
   [30/100] 완료
   [40/100] 완료
   [50/100] 완료
   ⚠️ 새 이미지 로딩 중... (1/7)
   ⚠️ 새 이미지 로딩 중... (1/7)
   ⚠️ 새 이미지 로딩 중... (1/7)
   [60/100] 완료
   ⚠️ 새 이미지 로딩 중... (1/7)
   ⚠️ 새 이미지 로딩 중... (1/7)
   ⚠️ 새 이미지 로딩 중... (1/7)
   ⚠️ 새 이미지 로딩 중... (2/7)
   ⚠️ 새 이미지 로딩 중... (3/7)
   ⚠️ 새 이미지 로딩 중... (1/7)
   ⚠️ 새 이미지 로딩 중... (2/7)
   ⚠️ 새 이미지 로딩 중... (3/7)
   ⚠️ 새 이미지 로딩 중... (4/7)
   ⚠️ 새 이미지 로딩 중... (5/7)
   ⚠️ 새 이미지 로딩 중... (1/7)
   ⚠️ 새 이미지 로딩 중... (2/7)
   [70/100] 완료
   ⚠️ 새 이미지 로딩 중... (1/7)
   ⚠️ 새 이미지 로딩 중... 